## 건강검진정보 데이터를 이용한 시각화 실습

In [ ]:
import pandas as pd
import platform
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager

if platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.rc('font', family=font_name)
elif platform.system() == 'Darwin':
    plt.rc('font',family='AppleGothic')
elif platform.system() == 'Linux': #When Using Colab
    !sudo apt-get install -y fonts-nanum
    !sudo fc-cache -fv
    !rm ~/.cache/matplotlib -rf
    plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

원본 데이터

https://www.data.go.kr/data/15007122/fileData.do

In [ ]:
df = pd.read_csv('./health_info.csv',encoding="cp949")
df

### 연령대, 성별에 따른 평균 신장 시각화
2개 항목에 대한 비교 - Bar Chart

In [ ]:
#연령대,성별에 따른 평균 신장 시각화

age_list = []
gender_list = []
height_mean_list =[]

sr=df.groupby(["연령대","성별"])["신장(5Cm단위)"].mean()

for idx in sr.index:
    age_list.append(idx[0])
    gender_list.append(idx[1])

for value in sr.values:
    height_mean_list.append(value)
    
df3 = pd.DataFrame({
    "연령대":age_list,
    "성별":gender_list,
    "평균신장":height_mean_list
})


plt.figure(figsize=(30, 15))
sns.barplot(x="연령대", y="평균신장", hue="성별", data=df3)
plt.ylim([140,175])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel("평균신장", fontsize=20)
plt.xlabel("연령대", fontsize=20)


### BMI와 허리둘레에 따른 데이터 분포 시각화
2개 항목에 대한 데이터 분포 - Scatter Chart

In [ ]:
#이상치 제거
cond = (df["허리둘레"] < 40) 
df = df.drop(df[cond].index, axis=0)
df.describe()

x = df["BMI"].values
y = df["허리둘레"].values

plt.figure(figsize=(30,20)) 
plt.xticks(fontsize=20)
plt.xlabel("BMI", fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel("허리둘레", fontsize=20)
plt.scatter(x,y)
plt.show()

### 비만인 가운데 혈압 분류별 비율을 시각화
전체 중 일부의 비율 표시 - Pie Chart(원그래프)

In [ ]:
cond = df["BMI"] < 20
df.loc[cond, "BMI구분"] = "저체중"

cond = (df["BMI"] >= 20) & (df["BMI"] < 25)
df.loc[cond, "BMI구분"] = "정상"

cond = (df["BMI"] >= 25) & (df["BMI"] < 30)
df.loc[cond, "BMI구분"] = "과체중"

cond = (df["BMI"] >= 30) 
df.loc[cond, "BMI구분"] = "비만"

df['BMI구분'].value_counts()
df

In [ ]:
#원그래프 시각화

cond = (df["BMI구분"] == "비만")
df2 = df[cond]

bp_types = df2['혈압분류'].value_counts().index.values
type_counts = df2['혈압분류'].value_counts().values
print(bp_types)
ratios = []
for count in type_counts:
    ratios.append(count/len(df2))

plt.figure(figsize=(10,6))    
plt.pie(ratios, labels=bp_types, autopct='%.2f%%')


### 각 연령대별 혈압 비율 시각화
어떻게 시각화할까? Bar Chart? Pie Chart?

In [ ]:
age_type_list = df["연령대"].unique()
#bp_type_list = df["혈압분류"].unique()
bp_type_list = ['정상','주의1','주의2','고혈압']
count_list = []

for age_type in age_type_list:
    cond = df["연령대"] == age_type
    sr = (df[cond])["혈압분류"].value_counts()
    total = sr.sum()
    tmp_list = []
    for bp_type in bp_type_list:
        ratio = (sr[bp_type]/total)*100
        tmp_list.append(ratio.round(1))
    count_list.append(tmp_list)

df2 = pd.DataFrame(count_list, index=age_type_list, columns=bp_type_list)

df3 = (df2.sort_index()).T.iloc[::-1]
plt.figure(figsize=(16,10))
sns.heatmap(df3, annot=True, fmt= '.1f', cmap="YlGnBu",yticklabels='auto')
#df3.plot.bar(stacked=True)

### 지역별 고혈압 인구 비율 시각화
Folium을 이용한 지도 시각화 활용

In [ ]:
!pip install folium

In [ ]:
import json
import folium


region_list = df["지역"].unique()
ratio_list = []

for region in region_list:
    cond = (df["지역"] == region)
    total = len(df[cond])
    high_bp_num = (df[cond]["혈압분류"].value_counts())["고혈압"]
    ratio = high_bp_num/total*100
    ratio_list.append(ratio.round(2))

df2 = pd.DataFrame({"지역명":region_list, "고혈압비율":ratio_list})


geojson_file = './map_korea.geojson'
geo_json = json.load(open(geojson_file, encoding='utf-8'))


for i in range(len(df2)):
    tooltip_text = df2['지역명'][i]+" : "+ str(df2['고혈압비율'][i])+"%"
    for j in range(len(geo_json['features'])):
        if geo_json['features'][j]['properties']['CTP_KOR_NM'] == df2['지역명'][i]:
            geo_json['features'][j]['properties']['tooltip1'] = tooltip_text

korea_map = folium.Map(
    location = [36.654957, 127.769162],   #좌표 값은 구글맵으로 구한다!
    tiles = 'Stamen Terrain',
    zoom_start = 7
)

choropleth=folium.Choropleth(
    geo_data = geo_json,
    data = df2,
    columns = ['지역명','고혈압비율'],
    fill_color='YlOrRd',
    key_on='properties.CTP_KOR_NM',
    highlight=True
).add_to(korea_map)


folium.LayerControl().add_to(korea_map)
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['tooltip1'], labels=False)
)

korea_map